***GENERATED CODE FOR likelihoodofdenial PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = transformationDF.drop('DateofService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17545', 'mean': '86390.13', 'stddev': '22202.31', 'min': '11011', 'max': 'RN001', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17545', 'mean': '7.3', 'stddev': '19.24', 'min': '0000', 'max': 'T6', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17545', 'mean': '', 'stddev': '', 'min': 'A41.9', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17545', 'mean': '', 'stddev': '', 'min': 'B02.9', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17545', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'U', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17545', 'mean': '', 'stddev': '', 'min': '<Unspecified>', 'max': 'Spine And Pelvis  Xrays', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17545', 'mean': '', 'stddev': '', 'min': 'Nurse Practitioner NP', 'max': 'Unspecified', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '17545', 'mean': '', 'stddev': '', 'min': '010B0F94-A0E7-4678-8214-CAA9EE3FFC26', 'max': 'FEE9B52E-00AE-4C56-9BCF-00899A963994', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature_label': 'PayerId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerId')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split


def gradientboostingclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    model_gbm_sklearn = GradientBoostingClassifier(**optuna_parameters)
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    model_gbm_sklearn.fit(X_train, y_train)
    display(" Accuracy of Model : %s" %
            model_gbm_sklearn.score(X_test, y_test))

    data = {'model': model_gbm_sklearn,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run likelihoodofdenialHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_OAGR', 'optionalDB': 'NextGen_OAGR', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT * FROM\r\n(\r\n\t\t\tSELECT DISTINCT\r\n\t\t\tcd.ChargeDetailID, \r\n\t\t\tcd.DateofService, \r\n\t\t\tcd.Amount,\r\n\t\t\tcd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\n\t\t\tcptc.CPTCode,\r\n\t\t\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\t\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\t\tIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\n\t\t\tIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\n\t\t\tMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END),\r\n\t\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\t\t\td.DepartmentCategory,\r\n\t\t\ts.SpecialityCategory,\r\n\t\t\tlabel = 0 , \r\n\t\t\tt.PayerId\r\n\t\t\tfrom ChargeDetail cd\r\n\t\t\tinner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n\t\t\tleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = e.PayerID  \r\n\t\t\tinner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\n\t\t\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\t\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\t\tWHERE\r\n\t\t\tcd.DateOfService between '1/01/2020' and '11/30/2020' \r\n\t\t\tand cd.Amount >.01 and t.CloseDate is not null\r\n\t\t\tand trc.reasonType = 'Denial'  \r\n\t\t\tand tt.TransactionTypeCategory = 'Charges'\r\n\t\t\t\r\nUNION\r\n\r\nSELECT DISTINCT \r\ncd.ChargeDetailID, cd.DateOfService,  \r\ncd.Amount,\r\ncd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal,\r\ncptc.CPTCode,\r\nCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\ndc.DiagnosisCode as pdiagnosis1,\r\ndc2.DiagnosisCode as pdiagnosis2,\r\nIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END), \r\nIsauthorized=(SELECT Case WHEN (apa.[AuthorizationCode]  IS NOT NULL) THEN 1 ELSE 0 END),   \r\nPredeterminationIndicator=(SELECT Case WHEN (apa.[PredeterminationIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\nMedNecessityRequiredIndicator=(SELECT Case WHEN (apa.[MedNecessityRequiredIndicator]  IS NOT NULL) THEN 1 ELSE 0 END), \r\nFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\nd.DepartmentCategory,\r\ns.SpecialityCategory,\r\nlabel = 1,\r\nt.PayerId\r\nfrom ChargeDetail cd\r\ninner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\nleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerID = e.PayerID\r\ninner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID  \r\ninner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID\r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\ninner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\ninner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\ninner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\ninner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\ninner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\nWHERE\r\ncd.DateOfService between '1/01/2020' and '11/30/2020'\r\nand cd.Amount >.01 and t.CloseDate is not null \r\nand trc.reasonType != 'Denial' \r\nand tt.TransactionTypeCategory = 'Charges' \r\n) as DND", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run likelihoodofdenialHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "17545", "mean": "", "stddev": "", "min": "00061C4A-9A47-4AE7-AF4B-173698E1456D", "max": "FFFEA6D5-80C3-4CB9-B696-C81C665BB070", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "DateofService", "transformation_label": "Extract Date"}], "feature": "DateofService", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "DateofService"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "17545", "mean": "454.65", "stddev": "1292.42", "min": "3.0", "max": "20898.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "17545", "mean": "4.06", "stddev": "103.88", "min": "-5703.0", "max": "5805.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PatientBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "17545", "mean": "26.31", "stddev": "397.34", "min": "-2215.31", "max": "20898.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InsuranceBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "17545", "mean": "26.42", "stddev": "396.91", "min": "-2215.31", "max": "20898.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PrimaryInsuranceBal"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17545", "mean": "86390.13", "stddev": "22202.31", "min": "11011", "max": "RN001", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17545", "mean": "7.3", "stddev": "19.24", "min": "0000", "max": "T6", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17545", "mean": "", "stddev": "", "min": "A41.9", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17545", "mean": "", "stddev": "", "min": "B02.9", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsBillable", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17545", "mean": "0.95", "stddev": "0.21", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "IsBillable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Isauthorized", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17545", "mean": "0.43", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Isauthorized"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PredeterminationIndicator", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17545", "mean": "0.43", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "PredeterminationIndicator"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MedNecessityRequiredIndicator", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17545", "mean": "0.43", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "MedNecessityRequiredIndic..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17545", "mean": "56.95", "stddev": "17.16", "min": "1", "max": "102", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17545", "mean": "", "stddev": "", "min": "F", "max": "U", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17545", "mean": "", "stddev": "", "min": "<Unspecified>", "max": "Spine And Pelvis  Xrays", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17545", "mean": "", "stddev": "", "min": "Nurse Practitioner NP", "max": "Unspecified", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "label", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "17545", "mean": "0.86", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "label"}, {"transformationsData": [{"feature_label": "PayerId", "transformation_label": "String Indexer"}], "feature": "PayerId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "17545", "mean": "", "stddev": "", "min": "010B0F94-A0E7-4678-8214-CAA9EE3FFC26", "max": "FEE9B52E-00AE-4C56-9BCF-00899A963994", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerId"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "17545", "mean": "24.64", "stddev": "49.96", "min": "0.0", "max": "433.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "CptModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "17545", "mean": "1.77", "stddev": "3.18", "min": "0.0", "max": "38.0", "missing": "0"}, "updatedLabel": "CptModifier_stringindexer..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "17545", "mean": "153.67", "stddev": "283.22", "min": "0.0", "max": "1587.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "17545", "mean": "140.63", "stddev": "267.42", "min": "0.0", "max": "1548.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "17545", "mean": "0.43", "stddev": "0.5", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "17545", "mean": "1.8", "stddev": "2.63", "min": "0.0", "max": "18.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "17545", "mean": "0.67", "stddev": "0.78", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PayerId_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "17545", "mean": "13.71", "stddev": "29.24", "min": "0.0", "max": "251.0", "missing": "0"}, "updatedLabel": "PayerId_stringindexer_tra..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run likelihoodofdenialHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=gradientboostingclassifier(autofe, ["Amount", "PatientBal", "InsuranceBal", "PrimaryInsuranceBal", "CPTCode_stringindexer", "CptModifier_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "IsBillable", "Isauthorized", "PredeterminationIndicator", "MedNecessityRequiredIndicator", "Age", "Gender_stringindexer", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "PayerId_stringindexer"], "label")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
